In [ ]:
import set_path

import numpy as np
import laspy
import geopandas as gpd

import open3d as o3d

from upcp.utils import ahn_utils, clip_utils

from gvl.helper_functions import color_clusters
from gvl.tree_detector import DetectorTree

In [ ]:
import warnings  # temporary, to supress deprecationwarnings from shapely
warnings.filterwarnings('ignore')

In [ ]:
# Use AHN subtiles
tilecode = '25DN2_20'

las_file = f'../datasets/pointcloud/tree_{tilecode}.laz'

In [ ]:
las = laspy.read(las_file)
pts = np.vstack((las.x, las.y, las.z)).T

In [ ]:
points_dtype = np.dtype([('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'),
                         ('Classification', '<i4')])
n = len(las.x)
points = np.empty(shape=(n,), dtype=points_dtype)

In [ ]:
points['X'] = pts[:,0]
points['Y'] = pts[:,1]
points['Z'] = pts[:,2]
points['Classification'] = las.classification # TODO label

In [ ]:
tree = DetectorTree(points[las.label == 1])

In [ ]:
# first clustering step
# tree.hdbscan_on_points(min_cluster_size=30, min_samples=10, xyz=False)
tree.hdbscan_on_points(min_cluster_size=50, min_samples=10, xyz=False)
tree.convex_hullify(points=tree.clustered_points)
# df_to_pg(tree.tree_df, schema='bomen', table_name='xy_bomen')

In [ ]:
tree.tree_df.plot(column='xy_clusterID', cmap='tab20')

In [ ]:
# second cluster step
tree.find_points_in_polygons(tree.tree_df)
# tree.kmean_cluster(tree.xy_grouped_points, round_val=2, min_dist=1)
tree.kmean_cluster(tree.xy_grouped_points, round_val=1, min_dist=3)
tree.convex_hullify(tree.kmean_grouped_points, kmean_pols=True)
# df_to_pg(tree.tree_df, schema='bomen', table_name='km_bomen')

In [ ]:
tree.tree_df.plot(column='xy_clusterID', cmap='tab20')

In [ ]:
tree.tree_df

In [ ]:
data = tree.kmean_grouped_points.to_records()

In [ ]:
# Shuffle classification for better visualisation
labels = np.unique(data['Classification'])
new_labels = labels.copy()

rng = np.random.default_rng()
rng.shuffle(new_labels)

shuffle_dict = {l: n for (l, n) in zip(labels, new_labels)}

shuffle_labels = np.array([shuffle_dict[l] for l in data['Classification']])

In [ ]:
header = laspy.LasHeader(point_format=3, version="1.2")
header.offsets = las.header.offsets
header.scales = las.header.scales

new_las = laspy.LasData(header)

new_las.x = data['X']
new_las.y = data['Y']
new_las.z = data['Z']

new_las.add_extra_dim(laspy.ExtraBytesParams(name="label", type="uint16",
                                         description="Label"))
new_las.label = shuffle_labels

In [ ]:
new_las.write(f'trees_{tilecode}_2.laz')